# Tensorflow Object Detection API and AWS Sagemaker

In [4]:
#ssd_resnet50_v1_fpn

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [5]:
%%capture
%pip install tensorflow_io sagemaker -U

In [6]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [7]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::359346771395:role/service-role/AmazonSageMaker-ExecutionRole-20240516T125452


In [8]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://udacity-selfdriving-240516-1358/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the repository
* build the docker image and push it 
* print the container name

In [9]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [ ]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.
#image_name = 'tf2-object-detection'
#!sh ./docker/build_and_push.sh $image_name

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [14]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

359346771395.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20240527120237


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [15]:
%%bash
pwd
rm -r /tmp/checkpoint
rm -r source_dir/checkpoint
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint

#wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
#tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

#ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8
#wget -O /tmp/ssdmobilenet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/ssdmobilenet.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint
# result out of memory with pipeline_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config
# will also test with: SSD_MobileNet_from_udacity_mentor.config

# Faster R-CNN ResNet152 V1 640x640
#wget -O /tmp/fasterrcnn.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/fasterrcnn.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint

#SSD ResNet50 V1 FPN 640x640 (RetinaNet50)
wget -O /tmp/ssdresnet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/ssdresnet50.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint

# Faster R-CNN ResNet50 V1 640x640
#wget -O /tmp/fasterrcnn50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/fasterrcnn50.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint


/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training


--2024-06-06 11:59:33--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.115.207, 172.253.62.207, 142.251.167.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.115.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘/tmp/ssdresnet50.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 12.8M 18s
    50K .......... .......... .......... .......... ..........  0% 23.9M 14s
   100K .......... .......... .......... .......... ..........  0% 22.9M 13s
   150K .......... .......... .......... .......... ..........  0% 19.3M 13s
   200K .......... .......... .......... .......... ..........  0% 23.1M 12s
   250K .......... .......... .......... .......... ..........  0% 23.8M 12s
   300K .......... .......... .......... .......... 

  5100K .......... .......... .......... .......... ..........  2% 44.1M 6s
  5150K .......... .......... .......... .......... ..........  2%  189M 6s
  5200K .......... .......... .......... .......... ..........  2% 38.8M 6s
  5250K .......... .......... .......... .......... ..........  2% 41.6M 6s
  5300K .......... .......... .......... .......... ..........  2% 42.3M 6s
  5350K .......... .......... .......... .......... ..........  2% 44.9M 6s
  5400K .......... .......... .......... .......... ..........  2%  116M 6s
  5450K .......... .......... .......... .......... ..........  2% 47.4M 6s
  5500K .......... .......... .......... .......... ..........  2% 43.5M 6s
  5550K .......... .......... .......... .......... ..........  2% 39.3M 6s
  5600K .......... .......... .......... .......... ..........  2% 51.1M 6s
  5650K .......... .......... .......... .......... ..........  2%  141M 6s
  5700K .......... .......... .......... .......... ..........  2% 41.0M 6s
  5750K ....

 10500K .......... .......... .......... .......... ..........  4%  362M 5s
 10550K .......... .......... .......... .......... ..........  4% 60.6M 5s
 10600K .......... .......... .......... .......... ..........  4% 61.9M 5s
 10650K .......... .......... .......... .......... ..........  4% 64.4M 5s
 10700K .......... .......... .......... .......... ..........  4% 80.9M 5s
 10750K .......... .......... .......... .......... ..........  4%  214M 5s
 10800K .......... .......... .......... .......... ..........  4% 63.1M 5s
 10850K .......... .......... .......... .......... ..........  4% 62.9M 5s
 10900K .......... .......... .......... .......... ..........  4% 67.0M 5s
 10950K .......... .......... .......... .......... ..........  4% 93.2M 5s
 11000K .......... .......... .......... .......... ..........  4%  201M 5s
 11050K .......... .......... .......... .......... ..........  4% 76.7M 5s
 11100K .......... .......... .......... .......... ..........  4% 69.9M 5s
 11150K ....

 15900K .......... .......... .......... .......... ..........  6%  337M 4s
 15950K .......... .......... .......... .......... ..........  6%  304M 4s
 16000K .......... .......... .......... .......... ..........  6%  337M 4s
 16050K .......... .......... .......... .......... ..........  6%  288M 4s
 16100K .......... .......... .......... .......... ..........  6%  336M 4s
 16150K .......... .......... .......... .......... ..........  6% 1.02M 5s
 16200K .......... .......... .......... .......... ..........  6%  399M 5s
 16250K .......... .......... .......... .......... ..........  6%  273M 5s
 16300K .......... .......... .......... .......... ..........  6%  372M 5s
 16350K .......... .......... .......... .......... ..........  6%  262M 5s
 16400K .......... .......... .......... .......... ..........  6%  228M 5s
 16450K .......... .......... .......... .......... ..........  6%  335M 5s
 16500K .......... .......... .......... .......... ..........  6%  131M 5s
 16550K ....

...

145500K .......... .......... .......... .......... .......... 60%  334M 1s
145550K .......... .......... .......... .......... .......... 60%  338M 1s
145600K .......... .......... .......... .......... .......... 60%  279M 1s
145650K .......... .......... .......... .......... .......... 60%  268M 1s
145700K .......... .......... .......... .......... .......... 60%  334M 1s
145750K .......... .......... .......... .......... .......... 60%  311M 1s
145800K .......... .......... .......... .......... .......... 61%  320M 1s
145850K .......... .......... .......... .......... .......... 61%  276M 1s
145900K .......... .......... .......... .......... .......... 61%  339M 1s
145950K .......... .......... .......... .......... .......... 61%  328M 1s
146000K .......... .......... .......... .......... .......... 61%  327M 1s
146050K .......... .......... .......... .......... .......... 61%  282M 1s
146100K .......... .......... .......... .......... .......... 61%  308M 1s
146150K ....

223850K .......... .......... .......... .......... .......... 93%  272M 0s
223900K .......... .......... .......... .......... .......... 93%  202M 0s
223950K .......... .......... .......... .......... .......... 93%  280M 0s
224000K .......... .......... .......... .......... .......... 93%  216M 0s
224050K .......... .......... .......... .......... .......... 93%  375M 0s
224100K .......... .......... .......... .......... .......... 93%  208M 0s
224150K .......... .......... .......... .......... .......... 93%  197M 0s
224200K .......... .......... .......... .......... .......... 93%  380M 0s
224250K .......... .......... .......... .......... .......... 93%  217M 0s
224300K .......... .......... .......... .......... .......... 93%  210M 0s
224350K .......... .......... .......... .......... .......... 93%  209M 0s
224400K .......... .......... .......... .......... .......... 93%  314M 0s
224450K .......... .......... .......... .......... .......... 93%  364M 0s
224500K ....

229250K .......... .......... .......... .......... .......... 95%  213M 0s
229300K .......... .......... .......... .......... .......... 95%  207M 0s
229350K .......... .......... .......... .......... .......... 95%  294K 0s
229400K .......... .......... .......... .......... .......... 95%  141M 0s
229450K .......... .......... .......... .......... .......... 95%  168M 0s
229500K .......... .......... .......... .......... .......... 96%  234M 0s
229550K .......... .......... .......... .......... .......... 96%  252M 0s
229600K .......... .......... .......... .......... .......... 96%  197M 0s
229650K .......... .......... .......... .......... .......... 96%  189M 0s
229700K .......... .......... .......... .......... .......... 96%  190M 0s
229750K .......... .......... .......... .......... .......... 96%  188M 0s
229800K .......... .......... .......... .......... .......... 96%  220M 0s
229850K .......... .......... .......... .......... .......... 96%  170M 0s
229900K ....

234650K .......... .......... .......... .......... .......... 98%  153M 0s
234700K .......... .......... .......... .......... .......... 98%  192M 0s
234750K .......... .......... .......... .......... .......... 98%  177M 0s
234800K .......... .......... .......... .......... .......... 98%  186M 0s
234850K .......... .......... .......... .......... .......... 98%  152M 0s
234900K .......... .......... .......... .......... .......... 98%  184M 0s
234950K .......... .......... .......... .......... .......... 98%  164M 0s
235000K .......... .......... .......... .......... .......... 98%  200M 0s
235050K .......... .......... .......... .......... .......... 98%  162M 0s
235100K .......... .......... .......... .......... .......... 98%  212M 0s
235150K .......... .......... .......... .......... .......... 98%  207M 0s
235200K .......... .......... .......... .......... .......... 98%  237M 0s
235250K .......... .......... .......... .......... .......... 98%  186M 0s
235300K ....

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir": "/opt/training",        
        "pipeline_config_path": "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.xlarge', # ml.g5.xlarge
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2024-06-06-12-00-02-576


2024-06-06 12:00:09 Starting - Starting the training job...
2024-06-06 12:00:25 Starting - Preparing the instances for training...
2024-06-06 12:00:57 Downloading - Downloading input data...
2024-06-06 12:01:17 Downloading - Downloading the training image.........
2024-06-06 12:02:57 Training - Training image download completed. Training in progress..2024-06-06 12:03:12,074 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 12:03:12,110 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 12:03:12,146 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 12:03:12,162 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "current_insta

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0606 12:03:18.270278 140205245101888 mirrored_strategy.py:419] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 2000
I0606 12:03:18.549876 140205245101888 config_util.py:552] Maybe overwriting train_steps: 2000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0606 12:03:18.550029 140205245101888 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0606 12:03:18.573611 140205245101888 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/object_detection/model_lib_v2.py:563: StrategyBase.experimental_distribute_datasets_from_function (from tensorflow.python.distribute.distribute_lib) is deprecated and will be removed in a future version.
Instructions for updating:
rename to distribute_datasets_from_function
INFO

I0606 12:04:02.368127 140199625475840 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I0606 12:04:07.362052 140199625475840 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I0606 12:04:12.386094 140199625475840 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]


In [17]:
# DEBUG / TROUBLESHOOT
#ls /tmp/checkpoint
#mkdir source_dir/checkpoint

import os

# Get the current working directory
current_directory = os.getcwd()
print(current_directory)

# Extract the directory name
directory_name = os.path.basename(current_directory)

# Print the directory name
print(directory_name)

files = [f for f in os.listdir('/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training/source_dir/checkpoint')]
for f in files:
    print(f)
    # do something

print("\n/opt")
files = [f for f in os.listdir('/opt/')]
for f in files:
    print(f)
    # do something

    
# /home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training/source_dir

/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training
1_model_training
ckpt-0.data-00000-of-00001
checkpoint
ckpt-0.index

/opt
aws
rh
containerd
amazon
aws-ofi-nccl
install-rootless-docker.sh
lib
brazil-npm-registry
docker.tgz
rootless.tgz
.sagemaker
ml


You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

mAP: 
